In [5]:
# Importing the libraries

#import pandas as pd 
#import numpy as np
#from sklearn.preprocessing import MinMaxScaler
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import GRU, Dense
#from tensorflow.keras.optimizers import Adam

In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from sklearn.metrics import mean_squared_error
import math

In [46]:
# Remover caracteres não numéricos e converter para float
def preprocess_value(value):
    try:
        value = float(''.join(c for c in str(value) if c.isdigit() or c == '.'))
    except ValueError:
        value = np.nan
    return value

In [47]:
# Carregar o dataset
dataframe = pd.read_csv('..\cleaned-data\ethereum-pools-fees-cleaned.csv')

In [48]:
# Aplicar pré-processamento para cada coluna no dataset
for column in dataframe.columns:
    dataframe[column] = dataframe[column].apply(preprocess_value)

# Converter o dataset para o tipo float32
dataset = dataframe.values.astype('float32')

In [49]:
# Normalizar o dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# Dividir em treino e teste
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

C:\Users\matheus.fidelis\AppData\Local\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\matheus.fidelis\AppData\Local\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [50]:
# Converter um array de valores em uma matriz de conjuntos de dados com base em observações anteriores.
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [51]:
# Reshape em X=t e Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [52]:
# Criar e ajustar a rede GRU
model = Sequential()
model.add(GRU(4, input_shape=(look_back, trainX.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)


Epoch 1/100
1197/1197 - 2s - loss: nan - 2s/epoch - 2ms/step
Epoch 2/100
1197/1197 - 1s - loss: nan - 967ms/epoch - 808us/step
Epoch 3/100
1197/1197 - 1s - loss: nan - 963ms/epoch - 804us/step
Epoch 4/100
1197/1197 - 1s - loss: nan - 961ms/epoch - 803us/step
Epoch 5/100
1197/1197 - 1s - loss: nan - 972ms/epoch - 812us/step
Epoch 6/100
1197/1197 - 1s - loss: nan - 967ms/epoch - 808us/step
Epoch 7/100
1197/1197 - 1s - loss: nan - 968ms/epoch - 809us/step
Epoch 8/100
1197/1197 - 1s - loss: nan - 967ms/epoch - 807us/step
Epoch 9/100
1197/1197 - 1s - loss: nan - 968ms/epoch - 808us/step
Epoch 10/100
1197/1197 - 1s - loss: nan - 951ms/epoch - 795us/step
Epoch 11/100
1197/1197 - 1s - loss: nan - 964ms/epoch - 806us/step
Epoch 12/100
1197/1197 - 1s - loss: nan - 957ms/epoch - 799us/step
Epoch 13/100
1197/1197 - 1s - loss: nan - 968ms/epoch - 809us/step
Epoch 14/100
1197/1197 - 1s - loss: nan - 952ms/epoch - 795us/step
Epoch 15/100
1197/1197 - 1s - loss: nan - 967ms/epoch - 808us/step
Epoch 16/

In [53]:
# Fazer as previsões
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Inverter as previsões
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

19/19 [==============================] - 0s 764us/step


ValueError: non-broadcastable output operand with shape (1197,1) doesn't match the broadcast shape (1197,17)

In [ ]:
# Calcular o erro quadrático médio da raiz
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))